In [1]:
# Disminuye la resolución espacial de datos horarios.

import pandas as pd
import xarray as xr

ds_1 = xr.open_dataset("../data/prec_hist_hist.nc")
ds_1["XLONG"] = ds_1["XLONG"].isel(south_north = 0)
ds_1["XLAT"] = ds_1["XLAT"].isel(west_east = 0)
ds_1 = ds_1.swap_dims( {"west_east": "XLONG", "south_north": "XLAT"} )
ds_1.coarsen( {"XLAT": 2, "XLONG": 2} ).mean().rename_dims(
    {"XLAT": "south_north", "XLONG": "west_east"}
    ).to_netcdf("../temp/prec_hist_comp_estaciones.nc")

In [7]:
# Hace compatible la resolución espacial de CHIRPS con WRF.

import pandas as pd
import xarray as xr

ds_1 = xr.open_dataset("../temp/prec_hist_hist_days.nc")
ds_1["XLONG"] = ds_1["XLONG"].isel(south_north = 0)
ds_1["XLAT"] = ds_1["XLAT"].isel(west_east = 0)

ds_1 = ds_1.swap_dims( {"west_east": "XLONG", "south_north": "XLAT"} )

ds_2 = xr.open_dataset( "../temp/CHIRPS_comp_hist.nc"
    ).rename_dims( {"time": "XTIME", "longitude": "XLONG",
    "latitude": "XLAT"} ).rename_vars( {"time": "XTIME",
    "longitude": "XLONG", "latitude": "XLAT", "precip": "Pcp"}
    ).interp( {"XLAT": ds_1["XLAT"], "XLONG": ds_1["XLONG"]},
    method = "cubic" ).rename_dims( {"XLAT": "latitude",
    "XLONG": "longitude", "XTIME": "time"} ).rename_vars( 
    {"Pcp": "precip", "XTIME": "time", "XLONG": "longitude",
    "XLAT": "latitude"} )

ds_2.to_netcdf("../temp/CHIRPS_interp_WRF.nc" )